In [1]:
import os
import sys
sys.path.append('/home/mark/Documents/code/mdrp')


import numpy as np
from utils.nlp_mdrp import mdrp_prob
import cyipopt



In [2]:
num_i = 5
num_j = 3

prob_setup = {}

prob_setup['s_time']   = np.array([[5,5,5,5,5],
                                   [5,5,5,5,5],
                                   [5,5,5,5,5]]).T/60
prob_setup['t_time']   = np.array([[45,30,10,10,10],
                                   [15,10,10,8,5],
                                   [20,25,30,20,20]]).T/60
prob_setup['cost']     = np.array([[12,12,12,12,12],
                                   [5,5,5,5,5],
                                   [5,5,5,5,5]]).T
prob_setup['beta']     = np.array([[0.5,0.5,0.5,0.5,0.5],
                                   [0.5,0.5,0.5,0.5,0.5],
                                   [0.5,0.5,0.5,0.5,0.5]]).T
prob_setup['N']        = np.ones(num_j)*10
prob_setup['k']        = np.ones(num_j)*10/60
prob_setup['mu']       = np.array([2,4,3])
prob_setup['max_rho']  = 0.9
prob_setup['max_cost'] = 10000
prob_setup['demand'] = 0.99

for param in prob_setup.keys():
    print('Parameter: %s \nValue:\n'%param,prob_setup[param])
    print('-----------------------------')

mdrp = mdrp_prob(prob_setup)

max_demand = mdrp._get_max_demand()
print('Max Demand: ',max_demand)


Parameter: s_time 
Value:
 [[0.08333333 0.08333333 0.08333333]
 [0.08333333 0.08333333 0.08333333]
 [0.08333333 0.08333333 0.08333333]
 [0.08333333 0.08333333 0.08333333]
 [0.08333333 0.08333333 0.08333333]]
-----------------------------
Parameter: t_time 
Value:
 [[0.75       0.25       0.33333333]
 [0.5        0.16666667 0.41666667]
 [0.16666667 0.16666667 0.5       ]
 [0.16666667 0.13333333 0.33333333]
 [0.16666667 0.08333333 0.33333333]]
-----------------------------
Parameter: cost 
Value:
 [[12  5  5]
 [12  5  5]
 [12  5  5]
 [12  5  5]
 [12  5  5]]
-----------------------------
Parameter: beta 
Value:
 [[0.5 0.5 0.5]
 [0.5 0.5 0.5]
 [0.5 0.5 0.5]
 [0.5 0.5 0.5]
 [0.5 0.5 0.5]]
-----------------------------
Parameter: N 
Value:
 [10. 10. 10.]
-----------------------------
Parameter: k 
Value:
 [0.16666667 0.16666667 0.16666667]
-----------------------------
Parameter: mu 
Value:
 [2 4 3]
-----------------------------
Parameter: max_rho 
Value:
 0.9
-----------------------------
P

In [3]:
x = mdrp.guess_init()

obj = mdrp.objective(x)
print(obj)

grad = mdrp.gradient(x)
print(grad)

cons = mdrp.constraints(x)
print(cons)

jac = mdrp.jacobian(x)
print(jac)

hess = mdrp.hessian(x,lagrange=None,obj_factor=1)
print(hess)



0.38174446612806073
[0.25990209 0.07189182 0.09648676 0.20990209 0.05522516 0.11315343
 0.14323542 0.05522516 0.1298201  0.14323542 0.04855849 0.09648676
 0.14323542 0.03855849 0.09648676]
[588.06      1.3365    0.66825   0.891    16.038    16.038    16.038
  16.038    16.038  ]
[[12.          5.          5.         12.          5.          5.
  12.          5.          5.         12.          5.          5.
  12.          5.          5.        ]
 [ 0.05        0.          0.          0.05        0.          0.
   0.05        0.          0.          0.05        0.          0.
   0.05        0.          0.        ]
 [ 0.          0.025       0.          0.          0.025       0.
   0.          0.025       0.          0.          0.025       0.
   0.          0.025       0.        ]
 [ 0.          0.          0.03333333  0.          0.          0.03333333
   0.          0.          0.03333333  0.          0.          0.03333333
   0.          0.          0.03333333]
 [ 1.          1.   

In [4]:
lb, ub, cl, cu = mdrp.get_bounds()
x0 = mdrp.guess_init()

nlp = cyipopt.Problem(
    n=len(x0),
    m=len(cl),
    problem_obj=mdrp,
    lb=lb,
    ub=ub,
    cl=cl,
    cu=cu
    )

# nlp.addOption('derivative_test', 'second-order')
nlp.add_option('mu_strategy', 'adaptive')
nlp.add_option('tol', 1e-7)

#
# Scale the problem (Just for demonstration purposes)
#
# nlp.set_problem_scaling(
#     obj_scaling=1,
#     x_scaling=[1, 1, 1, 1]
#     )
# nlp.add_option('nlp_scaling_method', 'user-scaling')

#
# Solve the problem
#
x, info = nlp.solve(x0)

print("Solution of the primal variables: x=%s\n" % repr(x))

print("Solution of the dual variables: lambda=%s\n" % repr(info['mult_g']))

print("Objective=%s\n" % repr(info['obj_val']))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:       75
Number of nonzeros in inequality constraint Jacobian.:       60
Number of nonzeros in Lagrangian Hessian.............:      120

Objective value at iteration #0 is - 0.381744
Total number of variables............................:       15
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       15
                     variables with only upper bounds:        0
Total number of equality constraints..........

In [5]:
def get_prices(x, mdrp, min_price, min_dph, max_dph):
    alpha = lambda a : (1/max_dph) + ((1/min_dph)-(1/max_dph))*a 
    # dollar per hour-->
    # 5$ per hour -> 1/5 0.2
    # 100$ per hour -> 1/100 0.01

    mat_x = np.round(mdrp._vec_to_mat(x)/mdrp.demand,3)
    # mat_x = mdrp._vec_to_mat(x)
    prices = np.zeros_like(mat_x)
    lat = mdrp._get_latency(x)
    for i in range(mdrp.n_orders):
        j_sort = np.argsort(-lat[i])
        # j starts at cheapest option (longest latency)
        prices[i,j_sort[0]] = min_price
        prev_j = j_sort[0]
        user_a = 1 - mat_x[i,j_sort[0]]
        for j in j_sort[1:]:
            prices[i,j] = prices[i,prev_j] + (lat[i,prev_j]-lat[i,j])/(alpha(user_a))
            user_a -= mat_x[i,j]
            prev_j = j

    return prices

def eval_sol(x,mdrp,mode_names):
    '''
    Inputs
        x          - solution as vector outputted by mdrp
        mdrp       - nlp_mdrp instance for the problem setting
                     contains useful methods
        mode_names - list of strings defining the name of the modes
    '''
    mat_x = mdrp._vec_to_mat(x)
    print('Problem Evlatuation')
    print('--------------'*4)
    demand = mdrp.demand*mdrp.n_orders
    print('Orders Demanded (order/hour): %.2f'%(demand))
    print('Maximum Cost ($/hour): %.2f'%mdrp.max_cost)
    print('--------------'*4)
    # Setup table
    print('Mode \t\t|', end="")
    for j in range(mdrp.n_modes):
        print('%s\t|'%mode_names[j],end="")
    print('%s\t|'%'total',end="")
    print('\n----------------|',end="")
    for _ in range(mdrp.n_modes+1):
        print('-------|',end="")
    # Orders
    orders_j = mat_x.sum(axis=0)/demand # sum over orders per mode
    print('\nOrders (%)\t|',end="")
    for j in range(mdrp.n_modes):
        print('%.2f\t|'%(orders_j[j]),end="")
    # tot_util = mat_x.sum(axis=0).sum(axis=0)/(mdrp.mu*mdrp.N).sum()
    print('%.2f\t|'%orders_j.sum(),end="")

    # Cost
    print('\nOp. Cost ($/hr)\t|',end="")
    for j in range(mdrp.n_modes):
        cost_j = np.inner(mdrp.cost[:,j],mat_x[:,j])
        print('%.2f\t|'%(cost_j),end="")
    tot_cost = np.inner(mdrp._mat_to_vec(mdrp.cost),x)
    print('%.2f\t|'%tot_cost,end="")

    # Utilization
    util = mdrp._get_util(x)
    print('\nUtilization (%)\t|',end="")
    for j in range(mdrp.n_modes):
        print('%.2f\t|'%util[j],end="")
    tot_util = mat_x.sum(axis=0).sum(axis=0)/(mdrp.mu*mdrp.N).sum()
    print('%.2f\t|'%tot_util,end="")
    
    # Delivery Time
    lat = mdrp._get_latency(x)*60
    print('\nD Time(min.)\t|',end="")
    for j in range(mdrp.n_modes):
        lat_j = np.inner(lat[:,j],mat_x[:,j])/(orders_j[j]*demand)
        print('%.2f\t|'%lat_j,end="")
    tot_lat = mdrp.objective(x)*60
    print('%.2f\t|'%tot_lat,end="")

    # Prices
    prices = get_prices(x, mdrp, 5, 5, 100)
    print('\nOrder Price\t|',end="")
    for j in range(mdrp.n_modes):
        price_j = np.inner(prices[:,j],mat_x[:,j])/(orders_j[j]*demand)
        print('%.2f\t|'%price_j,end="")
    tot_prices = np.inner(prices.flatten(),x)/(demand)
    print('%.2f\t|'%tot_prices,end="")
    # print("")
    # print(np.round(mat_x,2))
    # print(prices)

    # Price per hour
    pph = prices/(lat/60)
    # print(pph)
    print('\nPrice per hour\t|',end="")
    for j in range(mdrp.n_modes):
        price_j = np.inner(pph[:,j],mat_x[:,j])/(orders_j[j]*demand)
        print('%.2f\t|'%price_j,end="")
    tot_prices = np.inner(pph.flatten(),x)/(demand)
    print('%.2f\t|'%tot_prices,end="")

In [6]:
# print(lat)
eval_sol(x,mdrp,['car','drone','droid'])


Problem Evlatuation
--------------------------------------------------------
Orders Demanded (order/hour): 80.19
Maximum Cost ($/hour): 10000.00
--------------------------------------------------------
Mode 		|car	|drone	|droid	|total	|
----------------|-------|-------|-------|-------|
Orders (%)	|0.22	|0.45	|0.33	|1.00	|
Op. Cost ($/hr)	|216.00	|180.00	|130.95	|526.95	|
Utilization (%)	|0.90	|0.90	|0.87	|0.89	|
D Time(min.)	|15.20	|12.75	|25.25	|17.38	|
Order Price	|6.72	|6.59	|6.27	|6.52	|
Price per hour	|26.52	|31.93	|14.91	|25.16	|